# MVVM + Hexagonal in SEVA — Part 2 (View ↔ ViewModel ↔ UseCase)

This notebook teaches MVVM progressively. After each step, you run a complete mini app.

Flow:
1. View only
2. View + ViewModel
3. View + ViewModel + UseCase (port-based)
4. Swap to another view without changing VM/UseCase

In [ ]:
from pathlib import Path
import sys

# Allow notebook execution from docs/ while importing the project package.
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

## 1) Step A — View only (no button logic)

The view renders widgets and exposes `render(...)`. No workflow logic.

In [ ]:
import tkinter as tk
from tkinter import ttk


class StartViewOnly:
    """Pure View: widgets + render method, no business/use-case logic."""

    def __init__(self, root: tk.Tk):
        self.root = root
        self.root.title("MVVM Step A: View only")

        self.status_var = tk.StringVar(value="Idle")
        self.progress_var = tk.DoubleVar(value=0.0)

        ttk.Label(root, text="Experiment status").pack(padx=8, pady=(8, 2), anchor="w")
        ttk.Label(root, textvariable=self.status_var).pack(padx=8, pady=2, anchor="w")
        ttk.Progressbar(root, maximum=100, variable=self.progress_var, length=280).pack(padx=8, pady=6)

        # Intentionally no command binding yet.
        ttk.Button(root, text="Start").pack(padx=8, pady=(2, 8), anchor="w")

    def render(self, status_text: str, progress_pct: float) -> None:
        self.status_var.set(status_text)
        self.progress_var.set(progress_pct)

In [ ]:
# Step A assembled app (run this cell, close window to continue)
root = tk.Tk()
view = StartViewOnly(root)
view.render("View is ready", 0.0)
root.mainloop()

## 2) Step B — Add a ViewModel

This is a **didactic intermediate stage**: we keep UseCase out for one step so you can see what moves from View to ViewModel.

ViewModel business logic now does more than a raw assignment:
- validates if start is allowed,
- updates status transitions,
- applies deterministic progress rules.

In [ ]:
from dataclasses import dataclass


@dataclass
class StartVmState:
    status_text: str = "Idle"
    progress_pct: float = 0.0
    is_running: bool = False
    start_clicks: int = 0


class StartViewModelStepB:
    """ViewModel-only stage (no use case yet): state + command logic."""

    def __init__(self):
        self.state = StartVmState()

    def can_start(self) -> bool:
        # Business rule: do not restart while already running.
        return not self.state.is_running

    def on_start_clicked(self) -> None:
        if not self.can_start():
            self.state.status_text = "Already running"
            return

        self.state.is_running = True
        self.state.start_clicks += 1
        self.state.status_text = f"Run #{self.state.start_clicks} started"
        self.state.progress_pct = 0.0

    def on_poll_clicked(self) -> None:
        # Local demo rule: each poll adds +10% until completion.
        if not self.state.is_running:
            self.state.status_text = "Idle (start first)"
            return

        self.state.progress_pct = min(100.0, self.state.progress_pct + 10.0)
        self.state.status_text = f"Running... {self.state.progress_pct:.0f}%"

        if self.state.progress_pct >= 100.0:
            self.state.is_running = False
            self.state.status_text = "Done"

In [ ]:
def build_common_widgets(root: tk.Tk):
    """Small helper to reduce repeated Tkinter boilerplate in later steps."""
    status_var = tk.StringVar(value="Idle")
    progress_var = tk.DoubleVar(value=0.0)

    ttk.Label(root, text="Experiment status").pack(padx=8, pady=(8, 2), anchor="w")
    ttk.Label(root, textvariable=status_var).pack(padx=8, pady=2, anchor="w")
    ttk.Progressbar(root, maximum=100, variable=progress_var, length=280).pack(padx=8, pady=6)
    return status_var, progress_var


class StartViewWithVm:
    """View delegates events to VM and renders VM state."""

    def __init__(self, root: tk.Tk, vm: StartViewModelStepB):
        self.root = root
        self.vm = vm
        self.root.title("MVVM Step B: View + ViewModel")

        self.status_var, self.progress_var = build_common_widgets(root)

        row = ttk.Frame(root)
        row.pack(padx=8, pady=(2, 8), anchor="w")
        ttk.Button(row, text="Start", command=self._on_start).pack(side="left", padx=(0, 6))
        ttk.Button(row, text="Poll", command=self._on_poll).pack(side="left")

        self.render()

    def _on_start(self):
        self.vm.on_start_clicked()
        self.render()

    def _on_poll(self):
        self.vm.on_poll_clicked()
        self.render()

    def render(self):
        self.status_var.set(self.vm.state.status_text)
        self.progress_var.set(self.vm.state.progress_pct)

In [ ]:
# Step B assembled app (run this cell, click Start + Poll)
root = tk.Tk()
vm_b = StartViewModelStepB()
view_b = StartViewWithVm(root, vm_b)
root.mainloop()

## 3) Step C — Add a UseCase (now port-based)

Now we move workflow work into UseCase and keep VM focused on UI state orchestration.

Important hexagonal point:
- UseCase depends on **`JobPort` interface**, not on `JobRestMock`.
- We still inject `JobRestMock()` at runtime for offline demo.

In [ ]:
from seva.adapters.job_rest_mock import JobRestMock
from seva.domain.ports import JobPort


class StartAndPollProgressUseCase:
    """UseCase: start run + poll progress through the JobPort contract."""

    def __init__(self, job_port: JobPort):
        # Depends on port interface, not a concrete adapter implementation.
        self.job_port = job_port
        self.active_group_id = None

    def start(self) -> str:
        plan = JobRestMock.example_plan()
        group_id, _runs = self.job_port.start_batch(plan)
        self.active_group_id = group_id
        return group_id

    def poll_progress_pct(self) -> float:
        if not self.active_group_id:
            return 0.0

        snapshot = self.job_port.poll_group(self.active_group_id)

        # Demo simplification: read first run from first box.
        # In production code this is usually normalized into typed domain snapshots.
        for box in snapshot.get("boxes", {}).values():
            runs = box.get("runs", [])
            if runs:
                return float(runs[0].get("progress_pct", 0.0))
        return 0.0

In [ ]:
class StartViewModel:
    """ViewModel with richer UI business logic + use-case orchestration."""

    def __init__(self, use_case: StartAndPollProgressUseCase):
        self.use_case = use_case
        self.state = StartVmState()

    def can_start(self) -> bool:
        return not self.state.is_running

    def on_start_clicked(self) -> None:
        if not self.can_start():
            self.state.status_text = "Already running"
            return

        group_id = self.use_case.start()
        self.state.is_running = True
        self.state.start_clicks += 1
        self.state.progress_pct = 0.0
        self.state.status_text = f"Started group: {group_id}"

    def on_poll_clicked(self) -> None:
        if not self.state.is_running:
            self.state.status_text = "Idle (start first)"
            return

        progress = self.use_case.poll_progress_pct()
        self.state.progress_pct = progress

        if progress >= 100.0:
            self.state.status_text = "Done"
            self.state.is_running = False
        else:
            self.state.status_text = f"Running... {progress:.1f}%"

In [ ]:
class StartViewTk:
    """View only: event wiring + render. No use-case calls here."""

    def __init__(self, root: tk.Tk, vm: StartViewModel):
        self.root = root
        self.vm = vm
        self.root.title("MVVM Step C: View + ViewModel + UseCase")

        self.status_var, self.progress_var = build_common_widgets(root)

        row = ttk.Frame(root)
        row.pack(padx=8, pady=(2, 8), anchor="w")
        ttk.Button(row, text="Start", command=self._on_start).pack(side="left", padx=(0, 6))
        ttk.Button(row, text="Poll", command=self._on_poll).pack(side="left")

        self.render()

    def _on_start(self):
        self.vm.on_start_clicked()
        self.render()

    def _on_poll(self):
        self.vm.on_poll_clicked()
        self.render()

    def render(self):
        self.status_var.set(self.vm.state.status_text)
        self.progress_var.set(self.vm.state.progress_pct)

### About scheduling (`root.after`)

In this notebook, polling scheduling is in the **runner cell**, not in View or ViewModel.

Why?
- keeps View/UI testable and dumb,
- keeps ViewModel focused on state/commands,
- mirrors real apps where an App-Runner/Controller handles timing and lifecycle.

In [ ]:
# Step C assembled app (run this cell, click Start)
root = tk.Tk()
uc = StartAndPollProgressUseCase(JobRestMock())
vm_c = StartViewModel(uc)
view_c = StartViewTk(root, vm_c)

def _tick_poll():
    # Scheduler responsibility in runner/controller layer.
    if vm_c.state.is_running:
        vm_c.on_poll_clicked()
        view_c.render()
    root.after(1000, _tick_poll)

_tick_poll()
root.mainloop()

## 4) Swap the View (same ViewModel + UseCase)

Only View changes; VM and UseCase are untouched.

In [ ]:
class CompactStartViewTk:
    """Alternative View skin with same VM contract."""

    def __init__(self, root: tk.Tk, vm: StartViewModel):
        self.root = root
        self.vm = vm
        self.root.title("MVVM Step D: Compact View")

        self.label = ttk.Label(root, text="Idle")
        self.label.pack(padx=8, pady=(8, 4))

        self.progress = ttk.Progressbar(root, maximum=100, length=220)
        self.progress.pack(padx=8, pady=4)

        ttk.Button(root, text="Start", command=self._start).pack(padx=8, pady=2)
        ttk.Button(root, text="Poll", command=self._poll).pack(padx=8, pady=(2, 8))

        self.render()

    def _start(self):
        self.vm.on_start_clicked()
        self.render()

    def _poll(self):
        self.vm.on_poll_clicked()
        self.render()

    def render(self):
        self.label.configure(text=self.vm.state.status_text)
        self.progress["value"] = self.vm.state.progress_pct

In [ ]:
# Step D assembled app (same VM/UseCase, different View)
root = tk.Tk()
uc_d = StartAndPollProgressUseCase(JobRestMock())
vm_d = StartViewModel(uc_d)
view_d = CompactStartViewTk(root, vm_d)

def _tick_poll_d():
    if vm_d.state.is_running:
        vm_d.on_poll_clicked()
        view_d.render()
    root.after(1000, _tick_poll_d)

_tick_poll_d()
root.mainloop()

## 5) Short adapter cross-link

View/VM stay unchanged while adapters behind the UseCase are swapped:
- `JobRestMock()` (offline demo/testing)
- `JobRestAdapter(...)` (real API, e.g., Raspberry Pi host)
- `AmetekJobAdapter` (translate vendor semantics to `JobPort` method names)

In [ ]:
from seva.domain.entities import ExperimentPlan


class AmetekJobAdapter(JobPort):
    # Example skeleton: translate AMETEK semantics into JobPort contract.
    def start_batch(self, plan: ExperimentPlan):
        print("AMETEK: translate experiment -> vendor payload")
        return ("ametek-demo-group", {})

    def cancel_run(self, box_id, run_id):
        print("AMETEK: cancel run", box_id, run_id)

    def cancel_runs(self, box_to_run_ids):
        print("AMETEK: cancel runs", box_to_run_ids)

    def cancel_group(self, run_group_id):
        print("AMETEK: cancel group", run_group_id)

    def poll_group(self, run_group_id):
        print("AMETEK: poll", run_group_id)
        return {"boxes": {}, "wells": []}

    def download_group_zip(self, run_group_id, target_dir):
        print("AMETEK: download", run_group_id, target_dir)
        return target_dir